In [1]:
# Load Modules
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import ttest_ind
from scipy.stats import f_oneway
from math import pi



# Bokeh Library imports
import holoviews as hv
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool
from bokeh.palettes import d3
from bokeh.transform import factor_cmap

output_notebook()

Loading BokehJS ...

## Guided Question 1 

In [2]:
# Open Dataset and display info
clean_data = pd.read_csv('gapminder_clean.csv')
clean_data.head(10)

,Unnamed: 0,Country Name,Year,"Agriculture, value added (% of GDP)",CO2 emissions (metric tons per capita),Domestic credit provided by financial sector (% of GDP),Electric power consumption (kWh per capita),Energy use (kg of oil equivalent per capita),Exports of goods and services (% of GDP),"Fertility rate, total (births per woman)",GDP growth (annual %),Imports of goods and services (% of GDP),"Industry, value added (% of GDP)","Inflation, GDP deflator (annual %)","Life expectancy at birth, total (years)",Population density (people per sq. km of land area),"Services, etc., value added (% of GDP)",pop,continent,gdpPercap
0,0,Afghanistan,1962,NaN,0.073781,21.276422,NaN,NaN,4.878051,7.450,NaN,9.349593,NaN,NaN,33.219902,14.312061,NaN,10267083.0,Asia,853.100710
1,1,Afghanistan,1967,NaN,0.123782,9.917662,NaN,NaN,6.772908,7.450,NaN,14.209827,NaN,NaN,35.389415,15.881812,NaN,11537966.0,Asia,836.197138
2,2,Afghanistan,1972,NaN,0.130820,18.880833,NaN,NaN,14.763231,7.450,NaN,18.105850,NaN,NaN,37.610146,17.947027,NaN,13079460.0,Asia,739.981106
3,3,Afghanistan,1977,NaN,0.183118,13.836822,NaN,NaN,11.662904,7.449,NaN,14.823175,NaN,NaN,40.110146,19.998926,NaN,14880372.0,Asia,786.113360
4,4,Afghanistan,1982,NaN,0.165879,NaN,NaN,NaN,NaN,7.450,NaN,NaN,NaN,NaN,43.230732,19.402324,NaN,12881816.0,Asia,978.011439
5,5,Afghanistan,1987,NaN,0.275560,NaN,NaN,NaN,NaN,7.461,NaN,NaN,NaN,NaN,47.296341,17.366559,NaN,13867957.0,Asia,852.395945
6,6,Afghanistan,1992,NaN,0.101375,NaN,NaN,NaN,NaN,7.502,NaN,NaN,NaN,NaN,51.362927,21.054483,NaN,16317921.0,Asia,649.341395
7,7,Afghanistan,1997,NaN,0.060798,NaN,NaN,NaN,NaN,7.636,NaN,NaN,NaN,NaN,54.017829,27.623273,NaN,22227415.0,Asia,635.341351
8,8,Afghanistan,2002,38.471940,0.041129,NaN,NaN,NaN,32.386719,7.273,NaN,65.287704,23.714097,NaN,55.857195,32.912231,37.813963,25268405.0,Asia,726.734055
9,9,Afghanistan,2007,30.622854,0.087858,0.535181,NaN,NaN,17.823714,6.437,13.740205,58.350047,27.344703,22.382016,57.833829,39.637202,42.032443,31889923.0,Asia,974.580338


## Guided Question 2

In [3]:
# Filter data for 1962
data_1962 = clean_data[clean_data['Year'] == 1962]


# Initialise Plot
plot = figure(plot_width = 500, plot_height = 500, y_axis_type='log', x_axis_type='log', x_range=[0.01, 70], y_range=[100, 10 ** 5.3])
plot.title = 'The Correlation Between CO2 Emissions and GDP per Capita in 1962'
plot.yaxis.axis_label = 'Log(GDP per Capita)'
plot.xaxis.axis_label = 'Log(CO2 emissions (metric tons per capita))'
plot.ygrid.grid_line_alpha = 1
plot.xgrid.grid_line_alpha = 1


# Column Data Source
source = ColumnDataSource(data = {'emissions' : data_1962['CO2 emissions (metric tons per capita)'],
                                  'GDPpercap' : data_1962['gdpPercap'],
                                  'labels' : data_1962['Country Name']
                                 }
                         )


# Hover tool
hover = HoverTool(
        tooltips=[
            ("country", "@labels"),
            ("CO2 Emissions", "@emissions{0.00}"),
            ("GDP per Capita", "@GDPpercap{00.00}")
        ]
    )
                               

# Plot
plot.scatter('emissions', 'GDPpercap', size = 4, source=source)
plot.add_tools(hover)


show(plot)

## Guided Question 3

In [4]:
# Filter countries from dataset with NaN values for emissions or GDP per Cap
df = data_1962[data_1962['CO2 emissions (metric tons per capita)'].notnull() & data_1962['gdpPercap'].notnull()]

# Pearsons Rank Correlation test
x = df['CO2 emissions (metric tons per capita)']
y = df['gdpPercap']
correlation, Pvalue = pearsonr(x, y)
print("Pearson's r: {corr} (p value: {p})".format(corr = correlation, p = Pvalue))

Pearson's r: 0.9260816725019472 (p value: 1.1286792210038754e-46)


## Guided Question 4

In [5]:
df = clean_data[clean_data['CO2 emissions (metric tons per capita)'].notnull() & clean_data['gdpPercap'].notnull()]
stat = {}

# Collect data for each year
gdp = df.groupby('Year')['gdpPercap'].apply(list)
ems = df.groupby('Year')['CO2 emissions (metric tons per capita)'].apply(list)

# Run stats test for each year and generate dictionary
[stat.update({year : pearsonr(x, y)}) for year, x, y in zip(clean_data['Year'].unique(),ems, gdp)]

#
df = pd.DataFrame(stat,index = ['Correlation', 'p-value']).transpose()
df.sort_values(by = 'Correlation', ascending = False, inplace = True)
df

,Correlation,p-value
1967,0.938792,3.397143e-53
1962,0.926082,1.128679e-46
1972,0.842899,1.824292e-32
1982,0.816638,5.565916e-29
1987,0.809553,3.899627e-28
1992,0.809432,1.610614e-29
1997,0.808140,7.976156e-30
2002,0.800642,3.863564e-29
1977,0.792834,2.838892e-26
2007,0.720417,9.232747e-22


## Guided Question 5

In [28]:
# Filter countries from dataset with NaN values for emissions or GDP per Cap

data_1967 = clean_data[clean_data['Year'] == 1967]
data_1967 = data_1967[data_1967['CO2 emissions (metric tons per capita)'].notnull() & data_1967['gdpPercap'].notnull()]

# Intitialise Plot
plot = figure(plot_width = 750, plot_height = 750, y_axis_type='log', x_axis_type='log')
plot.xaxis.axis_label = 'CO2 emissions (metric tons per capita)'
plot.yaxis.axis_label = 'GDP per Capita'
plot.title = 'The Correlation between CO2 emissions and GDP per Capita in 1967'


#Data Source Column
source = ColumnDataSource(data = {'emissions' : data_1967['CO2 emissions (metric tons per capita)'],
                                  'GDPperCap' : data_1967['gdpPercap'],
                                  'countries' : data_1967['Country Name'], 
                                  'continent' : data_1967['continent'],
                                  'relative size' : (data_1967['pop']/data_1967['pop'].max())*500
                                 }
                         )


# Hover Tool 
hover = HoverTool(
        tooltips=[
            ("Country", "@countries"),
            ("CO2 Emissions", "@emissions{OO.OO}"),
            ("GDP per Capita", '@GDPperCap{OO.OO}')
        ]
    )


# Plot
plot.scatter(x ='emissions', 
             y= 'GDPperCap', 
             source = source,
             fill_color = factor_cmap('continent', factors = data_1967['continent'].unique(), palette = d3['Category10'][5] ),
            line_color = 'black',
            line_width = 0.75,
             line_alpha = 1,
            size = 'relative size',
            legend_group = 'continent',
             alpha = 0.5
            )

plot.add_tools(hover)
plot.legend.location = 'top_left'
plot.legend.title = 'Continent'
plot.legend.title_text_font_style = "bold"
plot.legend.title_text_font_size = "20px"
plot.legend.label_text_font_size = "16px"


show(plot)